In [62]:
from tensorflow.keras.applications.resnet50 import ResNet50
from keras.metrics import mean_squared_error
import keras
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten,Conv2D, ZeroPadding2D, Convolution2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
from tensorflow.keras.layers import MaxPooling2D
from keras.layers import Input, Concatenate, Conv2D, Flatten, Dense, BatchNormalization
from keras.models import Model
import tensorflow as tf
import cv2
import pandas as pd
import numpy as np
import os
from tensorflow.keras import activations


In [63]:
IMG_SIZE = 256
image_input = Input((IMG_SIZE, IMG_SIZE, 3), name='Image')
vector_input = Input((3), name='Numerical')

x = Conv2D(filters=3, kernel_size=(11, 11), strides=(2, 2), padding='valid', activation='relu')(image_input)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = BatchNormalization()(x)
x = Conv2D(filters=96, kernel_size=(5, 5), strides=(2, 2), padding='valid', activation='relu')(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = BatchNormalization()(x)
x = Conv2D(filters=192, kernel_size=(3, 3), strides=(2, 2), padding='valid', activation='relu')(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
resOut = BatchNormalization()(x)

flat_layer = Flatten()(resOut)
# print(resnet, vector_input)
# Concatenate the convolutional features and the vector input
concat_layer= Concatenate()([vector_input, flat_layer])
#final1 = Dense)(concat_layer)
#final2 = Dense(16)(concat_layer)
# dense1 = Dense(512,activation=activations.relu)(concat_layer)
dense2 = Dense(64, activation='relu')(concat_layer)
output = Dense(1, activation='sigmoid')(dense2)

# define a model with a list of two inputs
model = Model(inputs=[image_input, vector_input], outputs=output)

model.compile(
    optimizer="adam",
    loss='binary_crossentropy',
    metrics=['acc', tf.keras.metrics.AUC()]
)


In [64]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Image (InputLayer)              [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv2d_6 (Conv2D)               (None, 123, 123, 3)  1092        Image[0][0]                      
__________________________________________________________________________________________________
max_pooling2d_6 (MaxPooling2D)  (None, 61, 61, 3)    0           conv2d_6[0][0]                   
__________________________________________________________________________________________________
batch_normalization_6 (BatchNor (None, 61, 61, 3)    12          max_pooling2d_6[0][0]            
____________________________________________________________________________________________

In [65]:
from tensorflow.keras.utils import plot_model

## Dataframe

In [66]:
# ROOT_DIR = '/content/drive/My Drive'
# dataset = os.path.join(ROOT_DIR, 'ForKaggle')
cropped = '../input/cropped-data/Cropped_Data'
df_path = '../input/cropped-data/origa_g1020_refuge.csv'
data = pd.read_csv(df_path)
data.head()

,Image,Source,Cropped,CDR,Ecc-Cup,Ecc-Disc,Label
0,../input/cropped-data/Cropped_Data-20220623T20...,Origa,True,0.7097,0.636027,0.580909,0
1,../input/cropped-data/Cropped_Data-20220623T20...,Origa,True,0.6953,0.575024,0.608413,0
2,../input/cropped-data/Cropped_Data-20220623T20...,Origa,True,0.9629,0.299303,0.262828,0
3,../input/cropped-data/Cropped_Data-20220623T20...,Origa,True,0.7246,0.503186,0.483273,0
4,../input/cropped-data/Cropped_Data-20220623T20...,Origa,True,0.6138,0.500141,0.283467,0


**WARNING: There are missing values (very few i hope) in the Ecc-Cup column!!!**

In [67]:
for index, row in data.iterrows():
  if np.isnan(row['CDR']):
    data.at[index, 'CDR'] = 0.5
  if np.isnan(row['Ecc-Cup']):
    data.at[index, 'Ecc-Cup'] = 0.5

In [68]:
data.tail()

,Image,Source,Cropped,CDR,Ecc-Cup,Ecc-Disc,Label
2059,../input/refuge-cropped-data/train_cropped/ima...,REFUGE,True,0.535152,0.557495,0.539155,0
2060,../input/refuge-cropped-data/train_cropped/ima...,REFUGE,True,0.423169,0.259729,0.197245,0
2061,../input/refuge-cropped-data/train_cropped/ima...,REFUGE,True,0.483380,0.264281,0.550415,0
2062,../input/refuge-cropped-data/train_cropped/ima...,REFUGE,True,0.421299,0.306211,0.266774,0
2063,../input/refuge-cropped-data/train_cropped/ima...,REFUGE,True,0.443038,0.269389,0.448677,0


../input/cropped-data/Cropped_Data-20220623T201321Z-001/Cropped_Data/ORIGA/Images/001.jpg

In [69]:
data.head()

,Image,Source,Cropped,CDR,Ecc-Cup,Ecc-Disc,Label
0,../input/cropped-data/Cropped_Data-20220623T20...,Origa,True,0.7097,0.636027,0.580909,0
1,../input/cropped-data/Cropped_Data-20220623T20...,Origa,True,0.6953,0.575024,0.608413,0
2,../input/cropped-data/Cropped_Data-20220623T20...,Origa,True,0.9629,0.299303,0.262828,0
3,../input/cropped-data/Cropped_Data-20220623T20...,Origa,True,0.7246,0.503186,0.483273,0
4,../input/cropped-data/Cropped_Data-20220623T20...,Origa,True,0.6138,0.500141,0.283467,0


## Train Model

In [70]:
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(val_df, test_size=0.2, shuffle=True) # switch to shuffle=False later

In [71]:
g1020_inds = []
origa_inds = []
refuge_inds = []
for index, row in data.iterrows():
    if row['Source'] == 'G1020': g1020_inds.append(index)
    elif row['Source'] == 'Origa': origa_inds.append(index)
    elif row['Source'] == 'REFUGE': refuge_inds.append(index)

In [72]:
origa = data.loc[origa_inds]
g1020 = data.loc[g1020_inds]
refuge = data.loc[refuge_inds]

In [73]:
len(origa['Image'])

650

In [74]:
len(g1020['Image'])

1014

In [75]:
len(refuge['Image'])

400

In [76]:
TEST = 'REFUGE'
if TEST == 'REFUGE':
    train_df = origa.append(g1020)
    val_df = refuge
    
if TEST == 'ORIGA':
    train_df = g1020.append(refuge)
    val_df = origa

if TEST == 'G1020':
    train_df = origa.append(refuge)
    val_df = g1020

In [77]:
# origa = data.drop(g1020_inds, axis=0, inplace=False)
# g1020 = data.drop(origa_inds, axis=0, inplace=False)
# refuge = data.drop(refuge_inds, axis=0, inplace=False)

In [78]:
# # train_df = origa
# # val_df = g1020

# train_df = g1020
# val_df = origa

In [79]:
def generator(df):
  def callable_generator():
    for index, row in df.iterrows():
      loaded = load_img(row['Image'], target_size=(IMG_SIZE, IMG_SIZE))
      img_arr = np.array(loaded)/255.
      extracted_feats = np.array([row['CDR'], row['Ecc-Cup'], row['Ecc-Disc']])
      yield img_arr, extracted_feats, np.array([row['Label']]) # np.eye(2)[row['Label']]
  return callable_generator

In [80]:
train_dataset = tf.data.Dataset.from_generator(generator(train_df),
                                              (tf.float32, tf.float32, tf.float32),
                                              ((IMG_SIZE, IMG_SIZE, 3), (3,), (1,)))

In [81]:
BATCH_SIZE = 8
def prep_data(train_dataset):
    train_dataset = train_dataset.cache().batch(BATCH_SIZE).repeat()
    train_dataset = train_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
    return train_dataset

train_dataset = prep_data(train_dataset)

In [82]:
test_dataset = tf.data.Dataset.from_generator(generator(val_df),
                                              (tf.float32, tf.float32, tf.float32),
                                              ((IMG_SIZE, IMG_SIZE, 3), (3,), (1,)))
test_dataset = test_dataset.cache().batch(1).repeat()
test_dataset = test_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

In [83]:
from tensorflow.keras.losses import BinaryCrossentropy
from IPython.display import clear_output

In [84]:
tf.test.gpu_device_name()

2022-06-25 13:10:36.648469: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-25 13:10:36.649204: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-25 13:10:36.649763: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-25 13:10:36.650415: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-25 13:10:36.650990: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from S

'/device:GPU:0'

In [109]:
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix

In [86]:
# opt = tf.keras.optimizers.Adam(learning_rate=1e-2)
# bce = BinaryCrossentropy()
CLASS_WEIGHTS = None # {0: 0.4, 1: 0.6}
# @tf.function
def train_step(x, y):
    loss = model.train_on_batch(x, y, class_weight=CLASS_WEIGHTS)
    return loss

def train_model(n_epochs=1):
    end_epoch_accuracies = []
    end_epoch_aucs = []
    for e in range(n_epochs):
        for step, (a, b, y) in enumerate(train_dataset):
            if step == len(train_df['Image'])//BATCH_SIZE:
                break
            x = {'Image': a, 'Numerical': b}
            loss = train_step(x, y)
            clear_output(wait=True)
            print(step+1, loss)
        val_preds_cont = []
        val_preds_disc = []
        val_labels = []
        for step, (a, b, y) in enumerate(test_dataset):
            if step == len(val_df['Image']):
                break
            x = {'Image': a, 'Numerical': b}
            pred = model.predict(x)
            val_preds_cont.append(pred[0][0])
            val_preds_disc.append(int(pred[0][0] > 0.5))
            val_labels.append(int(float(y[0][0])))
#             print(val_preds_disc)
#             print(pred, 'Ground Truth:', y)
        acc = accuracy_score(val_labels, val_preds_disc)
        print(val_labels)
        try:
            roc = roc_auc_score(val_labels, val_preds_cont)
        except:
            roc = 0.5
        print(f'Accuracy: {acc}')
        print(f'AUC: {roc}')
        end_epoch_accuracies.append(acc)
        end_epoch_aucs.append(roc)
    return end_epoch_accuracies, end_epoch_aucs

In [87]:
accs, aucs = train_model(n_epochs=5)

208 [0.846269965171814, 0.375, 0.5999999642372131]
[0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 

In [92]:
accs

[0.83, 0.49, 0.6025, 0.9, 0.9075]

In [93]:
aucs

[0.6975694444444445,
 0.6536805555555556,
 0.605625,
 0.7320833333333333,
 0.8352777777777778]

In [94]:
sum(val_df['Label'])/len(val_df['Label'])

0.1

In [107]:
THRESH = 0.45

In [111]:
val_preds_cont = []
val_preds_disc = []
val_labels = []
for step, (a, b, y) in enumerate(test_dataset):
    if step == len(val_df['Image']):
        break
    x = {'Image': a, 'Numerical': b}
    pred = model.predict(x)
    val_preds_cont.append(pred[0][0])
    val_preds_disc.append(int(pred[0][0] > THRESH))
    val_labels.append(int(float(y[0][0])))
acc = accuracy_score(val_labels, val_preds_disc)
cf = confusion_matrix(val_labels, val_preds_disc)
try:
    roc = roc_auc_score(val_labels, val_preds_cont)
except:
    roc = 0.5
print(f'Accuracy: {acc}')
print(f'AUC: {roc}')
print(f'Confusion Matrix\n{cf}')


Accuracy: 0.93
AUC: 0.8352777777777778
Confusion Matrix
[[357   3]
 [ 25  15]]


In [112]:
model.save('refuge_test_model')

2022-06-25 13:29:44.354827: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


In [106]:
for step, (a, b, y) in enumerate(test_dataset):
  if step == 333:
    break
  x = {'Image': a, 'Numerical': b}
  pred = model.predict(x)
  print(int(pred[0][0]>THRESH), 'Ground Truth:', float(y[0][0]))

0 Ground Truth: 0.0
0 Ground Truth: 0.0
0 Ground Truth: 0.0
0 Ground Truth: 0.0
0 Ground Truth: 0.0
0 Ground Truth: 1.0
0 Ground Truth: 1.0
0 Ground Truth: 0.0
0 Ground Truth: 0.0
0 Ground Truth: 0.0
0 Ground Truth: 1.0
0 Ground Truth: 0.0
0 Ground Truth: 0.0
0 Ground Truth: 0.0
0 Ground Truth: 0.0
0 Ground Truth: 1.0
0 Ground Truth: 0.0
0 Ground Truth: 0.0
0 Ground Truth: 0.0
0 Ground Truth: 0.0
0 Ground Truth: 0.0
0 Ground Truth: 0.0
0 Ground Truth: 0.0
0 Ground Truth: 1.0
0 Ground Truth: 0.0
0 Ground Truth: 1.0
0 Ground Truth: 0.0
0 Ground Truth: 0.0
0 Ground Truth: 0.0
0 Ground Truth: 0.0
0 Ground Truth: 1.0
0 Ground Truth: 0.0
0 Ground Truth: 0.0
0 Ground Truth: 0.0
0 Ground Truth: 0.0
0 Ground Truth: 0.0
0 Ground Truth: 0.0
0 Ground Truth: 0.0
0 Ground Truth: 0.0
0 Ground Truth: 0.0
0 Ground Truth: 1.0
0 Ground Truth: 0.0
0 Ground Truth: 0.0
0 Ground Truth: 0.0
1 Ground Truth: 1.0
0 Ground Truth: 0.0
0 Ground Truth: 0.0
0 Ground Truth: 0.0
0 Ground Truth: 0.0
0 Ground Truth: 0.0
